In [11]:
import json
import requests
import configparser
import os

import pandas as pd
import numpy as np
import datetime, time
from sklearn import preprocessing
import logging


In [2]:
config = configparser.ConfigParser()
config.read('./credentials/dwh.cfg')

['./credentials/dwh.cfg']

In [3]:
os.environ['YELP_CLIENT_ID'] = config.get('YELP', 'YELP_CLIENT_ID')
os.environ['YELP_API_KEY'] = config.get('YELP', 'YELP_API_KEY')

In [4]:
yelp_url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(os.environ['YELP_API_KEY'])}

In [13]:
 category = 'Restaurants'
 districts  = [ 'Midtown West', 
                'Greenwich Village',
                'East Harlem',
                'Upper East Side',
                'Midtown East',
                'Gramercy',
                'Little Italy',
                'Chinatown',
                'SoHo',
                'Harlem',
                'Upper West Side',
                'Tribeca',
                'Garment District',
                'Stuyvesant Town',
                'Financial District',
                'Chelsea',
                'Morningside Heights',
                'Times Square',
                'Murray Hill',
                'East Village',
                'Lower East Side',
                'Hells Kitchen',
                'Central Park'
                ]

In [14]:
run_time = int(time.time())
for district in range(len(districts)):
    logging.info('-------------------\n Gathering data for {} \n ---------------------\n'.format(districts[district]))

    for step in range(20):
        location = districts[district] + ', Manhattan, NY'
        term = 'Restaurants'
        search_limit = 50
        offset = step*search_limit
        categories = '(restaurants, ALL)'
        sort_by = 'distance'

        url_params = {
                         'location': location.replace(' ', '+'),
                         'term' : term,
                         'limit': search_limit,
                         'offset': offset,
                         'categories': categories,
                         'sorty_by': sort_by
                     }
       
        response = requests.get(yelp_url, headers=headers, params=url_params)
        logging.info(('***** {} Restaurants #{} - #{} ....{}'.format(districts[district], 
                                                              offset+1, offset+search_limit,
                                                              response)))
        
        filename = 'restaurants-{}-{}.json'.format(districts[district], step)
        directory = './data/{}/'.format(str(run_time))

        os.makedirs(directory, exist_ok=True)

        try:
            data=response.json()
        except KeyError:
            continue

        with open(directory+filename, 'w') as data_object:
            json.dump(data, data_object)

        
logging.info(response)
logging.info(type(response.text))
logging.info(response.json().keys())
logging.info(response.text[:1000])

